In [ ]:
from dbase.database.SQLHelpers import store_SQL_data_Insert_Ignore, query_database
def save_previous_rates_date():
    import yfinance as yf
    print("Saving previous rates date")
    max_date = get_risk_free_rate_helper().index.max()
    rtes = yf.download('^IRX', progress=False, multi_level_index=False,start = max_date, end = (datetime.datetime.today()+ BDay(1)).strftime('%Y-%m-%d'), interval = '1h')
    print("DOWNLOAD COMPLETE")
    rtes.tz_convert('America/New_York')
    rtes.index = rtes.index.tz_convert('America/New_York')
    rtes.index = rtes.index.tz_localize(None)
    rtes.index = [x.replace(minute = 30) for x in rtes.index]
    rtes['annualized'] = rtes['Close']/100
    rtes['daily'] = (1 + rtes['annualized']) ** (1/365) - 1
    rtes['yf_tick'] = '^IRX'
    rtes['description'] = '	13 WEEK TREASURY BILL'
    rtes = rtes[rtes.index> get_risk_free_rate_helper().index.min()][['yf_tick', 'description', 'annualized', 'daily']]
    rtes.rename(columns = {'annualized': 'annualized_rate', 'daily': 'daily_rate', 'Date': 'datetime'}, inplace = True)
    rtes.index.name = 'datetime'
    rtes.reset_index(inplace = True)
    if rtes.empty:
        print("No new data to save")
        return
    store_SQL_data_Insert_Ignore('securities_master', 'rates_timeseries', rtes)
    return rtes

save_previous_rates_date()

In [1]:
import multiprocessing as mp
import queue

ctx = mp.get_context('fork')
manager = ctx.Manager()
_queue = manager.Queue()
queue.Empty
try:
    _queue.get(timeout=1)
except queue.Empty as e:
    print(e)
    print("Queue is empty")


Queue is empty


In [9]:
_queue.qsize()

0

In [ ]:
data_min = yf.download('^IRX', start='2025-05-01', end='2025-05-07', interval = '1h', progress=False, multi_level_index = False)
data_min.tz_convert('America/New_York')
data_min.index = data_min.index.tz_convert('America/New_York')
data_min.index = data_min.index.tz_localize(None)
data_min.index = [x.replace(minute = 30) for x in data_min.index]
data_min.columns = data_min.columns.str.lower()
data_min

,close,high,low,open,volume
2025-05-01 08:30:00,4.178,4.178,4.173,4.175,0
2025-05-01 09:30:00,4.178,4.178,4.170,4.173,0
2025-05-01 10:30:00,4.182,4.182,4.178,4.178,0
2025-05-01 11:30:00,4.182,4.188,4.178,4.178,0
2025-05-01 12:30:00,4.188,4.188,4.182,4.188,0
2025-05-01 13:30:00,4.185,4.188,4.185,4.188,0
2025-05-01 14:30:00,4.185,4.190,4.185,4.185,0
2025-05-02 08:30:00,4.200,4.200,4.175,4.182,0
2025-05-02 09:30:00,4.203,4.203,4.197,4.197,0
2025-05-02 10:30:00,4.205,4.205,4.197,4.197,0


In [ ]:
from dbase.DataAPI.ThetaData import resample
_rates_cache = None

In [ ]:


def deannualize(annual_rate, periods=365):
    return (1 + annual_rate) ** (1/periods) - 1


def get_risk_free_rate_helper(interval = '1d', use = 'db'):
    # download 3-month us treasury bills rates
    """
    Return timeseries of 3-month US treasury bills rates

    Parameters
    ----------
    interval : str
        Interval to resample the data to. Default is '1d'
    
    use: str
        Source of the data. Default is 'yf', other option is 'db'
    
    """

    data = _fetch_rates(interval = interval).copy()
    return resample(data, interval, {'daily':'last', 'annualized': 'last', 'name': 'last', 'description': 'last'})


def _fetch_rates(interval):
    """
    Handles _rates_cache logic picking
    """
    global _rates_cache
    ## First check data base.
    if _rates_cache is None:
        data = query_database('securities_master','rates_timeseries' ,"SELECT * FROM rates_timeseries WHERE yf_tick = '^IRX' AND DATETIME >= '2010-01-01'")
        data['datetime'] = pd.to_datetime(data['datetime'])
        data.set_index('datetime',inplace = True)
        data.rename(columns = {'daily_rate':'daily', 'annualized_rate':'annualized', 'yf_tick': 'name'}, inplace = True)
        data.index.name = 'Datetime'
    else:
        data = _rates_cache.copy()

    ## Now, if data is not up to date, update it
    if data.index.max().date() < change_to_last_busday(datetime.datetime.now()).date():
        # logger.info('Updating rates data')
        ## Query from max to today from retrieve_timeseries
        ## Prefer using yf because openbb timezone is UTC for IRX
        ## For YF, we have to do end_date + 1 day to get the last day
        data_min = yf.download('^IRX', data.index.max().date(), end = (datetime.datetime.today()+ BDay(1)).strftime('%Y-%m-%d') , interval = '1h', progress=False, multi_level_index = False)
        data_min.tz_convert('America/New_York')
        data_min.index = data_min.index.tz_convert('America/New_York')
        data_min.index = data_min.index.tz_localize(None)
        data_min.index = [x.replace(minute = 30) for x in data_min.index]
        data_min.columns = data_min.columns.str.lower()
        data_min['daily'] = data_min['close'].apply(deannualize)
        data_min['annualized'] = data_min['close']/100
        data_min['name'] = '^IRX'
        data_min['description'] = '13 WEEK TREASURY BILL'
        data_min.index.name = 'Datetime'
        data_min = data_min[['name', 'description', 'daily', 'annualized']]
        data = pd.concat([data, data_min])
    
        _rates_cache =  resample(data, '30m', {'daily':'ffill', 'annualized': 'ffill', 'name': 'ffill', 'description': 'ffill'})
    return data


In [ ]:
query_database('securities_master','rates_timeseries' ,"SELECT * FROM rates_timeseries WHERE yf_tick = '^IRX' AND DATETIME >= '2010-01-01'")

,datetime,daily_rate,annualized_rate,yf_tick,description
0,2010-01-01 09:30:00,0.000134,0.0005,^IRX,13 WEEK TREASURY BILL
1,2010-01-01 10:30:00,0.000134,0.0005,^IRX,13 WEEK TREASURY BILL
2,2010-01-01 11:30:00,0.000134,0.0005,^IRX,13 WEEK TREASURY BILL
3,2010-01-01 12:30:00,0.000134,0.0005,^IRX,13 WEEK TREASURY BILL
4,2010-01-01 13:30:00,0.000134,0.0005,^IRX,13 WEEK TREASURY BILL
...,...,...,...,...,...
46413,2025-05-06 10:30:00,0.000113,0.0421,^IRX,13 week treasury bill
46414,2025-05-06 11:30:00,0.000113,0.0421,^IRX,13 week treasury bill
46415,2025-05-06 12:30:00,0.000113,0.0421,^IRX,13 week treasury bill
46416,2025-05-06 13:30:00,0.000113,0.0421,^IRX,13 week treasury bill


In [ ]:
get_risk_free_rate_helper(interval = '1d')

,daily,annualized,name,description
Datetime,,,,
2010-01-01,0.000134,0.00050,^IRX,13 WEEK TREASURY BILL
2010-01-04,0.000147,0.00055,^IRX,13 WEEK TREASURY BILL
2010-01-05,0.000160,0.00060,^IRX,13 WEEK TREASURY BILL
2010-01-06,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
2010-01-07,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
...,...,...,...,...
2025-04-30,0.004522,0.04190,^IRX,13 WEEK TREASURY BILL
2025-05-01,0.004519,0.04185,^IRX,13 WEEK TREASURY BILL
2025-05-02,0.004531,0.04208,^IRX,13 WEEK TREASURY BILL


In [ ]:
from trade.assets.rates import get_risk_free_rate_helper
get_risk_free_rate_helper()

,daily,annualized,name,description
Datetime,,,,
2010-01-01,0.000134,0.00050,^IRX,13 WEEK TREASURY BILL
2010-01-04,0.000147,0.00055,^IRX,13 WEEK TREASURY BILL
2010-01-05,0.000160,0.00060,^IRX,13 WEEK TREASURY BILL
2010-01-06,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
2010-01-07,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
...,...,...,...,...
2025-04-30,0.004522,0.04190,^IRX,13 WEEK TREASURY BILL
2025-05-01,0.004519,0.04185,^IRX,13 WEEK TREASURY BILL
2025-05-02,0.004531,0.04208,^IRX,13 WEEK TREASURY BILL


In [ ]:
yf.download?

Signature:
yf.download(
    tickers,
    start=None,
    end=None,
    actions=False,
    threads=True,
    ignore_tz=None,
    group_by='column',
    auto_adjust=None,
    back_adjust=False,
    repair=False,
    keepna=False,
    progress=True,
    period='max',
    interval='1d',
    prepost=False,
    proxy=<object object at 0x12735b180>,
    rounding=False,
    timeout=10,
    session=None,
    multi_level_index=True,
) -> Optional[pandas.core.frame.DataFrame]
Docstring:
Download yahoo tickers
:Parameters:
    tickers : str, list
        List of tickers to download
    period : str
        Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        Either Use period parameter or use start and end
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        Intraday data cannot extend last 60 days
    start: str
        Download start date string (YYYY-MM-DD) or _datetime, inclusive.
        Default is 99 years ago
        E.g. for start="2020

In [ ]:
from dbase.database.SQLHelpers import store_SQL_data_Insert_Ignore, query_database
def save_previous_rates_date():
    import yfinance as yf
    print("Saving previous rates date")
    max_date = get_risk_free_rate_helper().index.max()
    rtes = yf.download('^IRX', progress=False, multi_level_index=False,start = max_date, end = (datetime.datetime.today()+ BDay(1)).strftime('%Y-%m-%d'), interval = '1h')
    print("DOWNLOAD COMPLETE")
    rtes.tz_convert('America/New_York')
    rtes.index = rtes.index.tz_convert('America/New_York')
    rtes.index = rtes.index.tz_localize(None)
    rtes.index = [x.replace(minute = 30) for x in rtes.index]
    rtes['annualized'] = rtes['Close']/100
    rtes['daily'] = (1 + rtes['annualized']) ** (1/365) - 1
    rtes['yf_tick'] = '^IRX'
    rtes['description'] = '	13 WEEK TREASURY BILL'
    rtes = rtes[rtes.index> get_risk_free_rate_helper().index.min()][['yf_tick', 'description', 'annualized', 'daily']]
    rtes.rename(columns = {'annualized': 'annualized_rate', 'daily': 'daily_rate', 'Date': 'datetime'}, inplace = True)
    rtes.index.name = 'datetime'
    rtes.reset_index(inplace = True)
    if rtes.empty:
        print("No new data to save")
        return
    store_SQL_data_Insert_Ignore('securities_master', 'rates_timeseries', rtes)
    return rtes

save_previous_rates_date()

Saving previous rates date
DOWNLOAD COMPLETE
Size to be inserted: 7


,datetime,yf_tick,description,annualized_rate,daily_rate
0,2025-05-06 08:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04213,0.000113
1,2025-05-06 09:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
2,2025-05-06 10:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
3,2025-05-06 11:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
4,2025-05-06 12:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
5,2025-05-06 13:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
6,2025-05-06 14:30:00,^IRX,\t13 WEEK TREASURY BILL,0.04210,0.000113
